Execute in a shell:

```
mkdir supercdms-data
alias ='aws s3 --profile slac_public --endpoint-url https://maritime.sealstorage.io/api/v0/s3 --no-verify-ssl'
s3 ls --recursive s3://utah/supercdms-data/CDMS/UMN/R68/Raw/  | awk '{print $4}' > supercdms-data/list.txt
```

In [21]:
import os,sys,boto3,json,xmltodict,itertools,subprocess
from botocore.client import Config
import midas.file_reader
from pprint import pprint
import numpy as np



# /////////////////////////////////////////
def GetBanksData(filename):
    reader = midas.file_reader.MidasFile(filename)
    for E,evt in enumerate(reader):
      for bank_name, bank in evt.banks.items():
        yield (evt, bank)

# ///////////////////////////////////////////////////////////////////
def LoadXML(filename):
	with open(filename, 'rt') as file: 
		body = file.read() 
	return 

# ///////////////////////////////////////////////////////////////////
def Merge(d1, d2):
    return {**d1, **d2} if d1 and d2 else (d1 or d2)

# ///////////////////////////////////////////////////////////////////
def Concat(v):
    return np.array(itertools.chain.from_iterable(v))

# ///////////////////////////////////////////////////////////////////
def PreviewBinary(obj):

    if obj is None or isinstance(obj, str) :
        return {
            "type": type(obj).__name__,
            "value": obj,
        }

    if isinstance(obj, bytes):
        return {
            "type": type(obj).__name__,
            "len": len(obj),
        }

    assert(isinstance(obj,(tuple,list,np.array)))
    return {
            "type": type(obj).__name__,
            "shape": obj.shape if hasattr(obj, "shape") else [len(obj)],
            "dtype": str(obj.dtype) if hasattr(obj, "dtype") else type(obj[0]).__name__,
            "vmin": str(np.min(obj)),
            "vmax": str(np.max(obj))
        }

    raise Exception("problem")

# /////////////////////////////////////////
def ParseEvent(evt):

    ret={}

    ret["header"]={
        "event_id": evt.header.event_id,  # int
        "trigger_mask": evt.header.trigger_mask,  # int
        "serial_number":evt.header.serial_number,  # int
        "timestamp": evt.header.timestamp,  # int UNIX timestamp of event
        "event_data_size_bytes": evt.header.event_data_size_bytes,  # int Size of all banks
    }
    ret["all_bank_size_bytes"]=evt.all_bank_size_bytes  # (int)
    ret["flags"]=evt.flags
    ret["non_bank_data"]=evt.non_bank_data # (bytes or None) - Content of some special events that don't
    
    ret["banks"]={}
    for bank_name, bank in evt.banks.items():
        ret["banks"][bank.name]={
            "name": bank.name,  # (str) - 4 characters
            "type": bank.type,  # (int) - See `TID_xxx` members in `midas` module
            "size_bytes": bank.size_bytes,  # (int)
            "data": bank.data # (tuple of int/float/byte etc, or a numpy array if use_numpy is specified when unpacking),
        }

    return ret

# ////////////////////////////////////////////////////////////////
def Shell(cmd):
	return subprocess.check_output(cmd, shell=True, text=True)

# /////////////////////////////////////////
def ConnectS3(bucket_name=None, endpoint_url=None,aws_access_key_id=None, aws_secret_access_key=None, signature_version = None):
    config = Config(signature_version = signature_version)
    s3 = boto3.resource('s3', endpoint_url=endpoint_url, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, config=config,)
    bucket = s3.Bucket(bucket_name)
    return bucket

# /////////////////////////////////////////
def ParseXML(body):
    d=xmltodict.parse(''.join(c for c in body if c.isprintable())) # I am loosing some char this
    assert(isinstance(d,dict))
    return d

    # /////////////////////////////////////////
def SaveArray(filename, data):
    os.makedirs(os.path.dirname(filename),exist_ok=True)
    np.savez_compressed(filename, data)

    # /////////////////////////////////////////
def SaveJSON(filename,d):
    os.makedirs(os.path.dirname(filename),exist_ok=True)
    with open(json_filename,"w") as out:
        out.write(d)


# ///////////////////////////////////////////////////////////////
bucket=ConnectS3("utah", 'https://maritime.sealstorage.io/api/v0/s3','any','any','s3v4')
max_files=1
pulses={}
    
with open("supercdms-data/list.txt","r") as f:
  files=[it.strip() for it in f.readlines() if it.strip().endswith(".mid.gz")]
print("found",len(files),".mid.gz files")
        
for I,key in enumerate(files):

    if max_files and I>=max_files:
        break
            
    key_noext=key.replace(".mid.gz","")

    # download file
    if not os.path.isfile(key):
        os.makedirs(os.path.dirname(key),exist_ok=True)
        bucket.download_file(key,key)
        print(f"Downloaded file {key} {os.path.getsize(key)}")
    else:
        # print(f"File {key} already exists {os.path.getsize(key)}")
        pass

    # generate json
    if True or not os.path.isfile(key_noext+".json"):
        reader = midas.file_reader.MidasFile(key)
        events=[]
        
        data={}
        for E,evt in enumerate(reader):
            parsed=ParseEvent(evt)
        
            # parse xml
            if E==0:
                odb=ParseXML(parsed["non_bank_data"].decode("latin-1"))
                assert(list(odb.keys())==["odb"])
                parsed["odb"]=odb['odb']
                del parsed["non_bank_data"]
            else:
                data[f"events_{E}_non_bank_data"]=parsed["non_bank_data"]
                parsed["non_bank_data"]=PreviewBinary(parsed["non_bank_data"])

            # streamable
            B=0
            for bank_name,bank in parsed["banks"].items():
                data[f"events_{E}_banks_{B}_data"]=bank["data"]
                bank["data"]=PreviewBinary(bank["data"]) # remove binary
                B+=1
            
            events.append(parsed)
        
        npz_filename=f"{key_noext}.npz"
        os.makedirs(os.path.dirname(npz_filename),exist_ok=True)
        np.savez_compressed(npz_filename, **data)

        # save json
        d=json.dumps(events, sort_keys=False, indent=2)
        json_filename=f"{key_noext}.json"
        SaveJSON(json_filename,d)

        # if you want compressed
        # uncompressed_size=os.path.getsize(json_filename)
        #Shell(f"gzip --keep --force {json_filename}")
        #compressed_size=os.path.getsize(json_filename + '.gz')
        #print("JSON size size", f"{uncompressed_size:,}",f"{compressed_size:,}")

    print("Done",key)

found 39522 .mid.gz files
Done supercdms-data/CDMS/UMN/R68/Raw/07180808_1558/07180808_1558_F0001.mid.gz


In [24]:
d=np.load("/home/scrgiorgio/github.com/nsdf-fabric/nsdf-slac/supercdms-data/CDMS/UMN/R68/Raw/07180808_1558/07180808_1558_F0001.npz")

for k,v in d.items():
  print(it)
  if "banks_" in it:
    print)
    break

events_1_non_bank_data
events_2_non_bank_data
events_3_non_bank_data
events_4_non_bank_data
events_5_non_bank_data
events_6_non_bank_data
events_7_non_bank_data
events_8_non_bank_data
events_8_banks_0_data
events_9_non_bank_data
events_10_non_bank_data
events_10_banks_0_data
events_11_non_bank_data
events_12_non_bank_data
events_13_non_bank_data
events_14_non_bank_data
events_15_non_bank_data
events_16_non_bank_data
events_17_non_bank_data
events_18_non_bank_data
events_19_non_bank_data
events_19_banks_0_data
events_20_non_bank_data
events_21_non_bank_data
events_22_non_bank_data
events_23_non_bank_data
events_24_non_bank_data
events_25_non_bank_data
events_26_non_bank_data
events_27_non_bank_data
events_28_non_bank_data
events_29_non_bank_data
events_29_banks_0_data
events_30_non_bank_data
events_31_non_bank_data
events_32_non_bank_data
events_33_non_bank_data
events_34_non_bank_data
events_35_non_bank_data
events_36_non_bank_data
events_37_non_bank_data
events_38_non_bank_data
events

In [ ]:


pulses=[]
for F,filename in enumerate(files[0:1]): # TODO ... signal otherwise is too big
    print("Reading",F, "filename", filename)
    for evt, bank in GetBanksData(filename):
        pulses.append(bank.data)

pulse=Concat(pulses)
del pulses
print("dtype",pulse.dtype,"shape", pulse.shape,"vmin",np.min(pulse),"vmax",np.max(pulse))

In [ ]:
import shutil
import numpy as np
import OpenVisus as ov
idx_filename='/mnt/c/big/visus-datasets/signal1d_slac/visus.idx'

N=pulse.shape[0]
shutil.rmtree(os.path.splitext(os.path.dirname(idx_filename))[0], ignore_errors=True)
db=ov.CreateIdx(
  url=idx_filename, 
  dims=[N],
  fields=[ov.Field('data',ov.DType.fromString(str(pulse.dtype)),'row_major')], 
  compression="raw", 
  arco=f"{4*1024*1024}")
assert(os.path.isfile(idx_filename))

print("Writing data...")
logic_box=logic_box=ov.BoxNi(ov.PointNi([0]),ov.PointNi([N]))
db.write(pulse,  logic_box=logic_box)
print("write uncompressed data done","logic_box",logic_box.toString())

In [ ]:
print("Compressing data")
db=ov.LoadDataset(idx_filename)
db.compressDataset("zip") 
print("compress dataset done")

Read data

In [ ]:
logic_box=db.getLogicBox()
print("logic_box",logic_box)
print("db.getMaxResolution()",db.getMaxResolution())
resolution=12
data=db.read(logic_box=logic_box, max_resolution=resolution)
print(f"IDX read done dtype={data.dtype} shape={data.shape} vmin={np.min(data)} vmax={np.max(data)}")

Show data

In [ ]:
import bokeh
import bokeh.io 
bokeh.io.output_notebook()

In [ ]:
from bokeh.plotting import figure, show
p = bokeh.plotting.figure(title="Simple line example", x_axis_label='x', y_axis_label='y')
y=data
x=list(range(len(y)))
p.line(x, y, legend_label="Pulse", line_width=1)
show(p)

Copy blocks to S3. For example:


```bash
aws s3 sync --no-verify-ssl --endpoint-url https://maritime.sealstorage.io/api/v0/s3 --profile sealstorage --size-only  C:/big/visus-datasets/signal1d_slac/ s3://utah/visus-datasets/signal1d_slac/
```

# With `max` filter

In [ ]:
idx_filename_max='/mnt/c/big/visus-datasets/signal1d_slac_max/visus.idx'
shutil.rmtree(os.path.splitext(idx_filename_max)[0], ignore_errors=True)
field=ov.Field('data',f"int64[2]",'row_major')
N=pulse.shape[0]

db=ov.CreateIdx(url=idx_filename_max, 
  dim=1, 
  dims=[N],
  fields=[field], 
  compression="raw",
  filters=['max'],
  arco=2*1024*1024
)
assert(os.path.isfile(idx_filename_max))

print(f"Source pulse dtype={pulse.dtype} shape={pulse.shape} vmin={np.min(pulse):,} vmax={np.max(pulse)}")
pulse_max=np.zeros((N, 2), dtype=pulse.dtype)
pulse_max[:,0]=pulse[:]
vmin,vmax=np.min(pulse_max[:,0]),np.max(pulse_max[:,0])
print(f"New pulse with extra channel shape={pulse_max.shape} dtype={pulse_max.dtype} vmin={vmin:,} vmax={vmax:,}")

print("Writing data...")
logic_box=ov.BoxNi(ov.PointNi([0]),ov.PointNi([N]))
db.write(pulse_max, logic_box=logic_box)
print(f"written max pulse logic_box=[{logic_box.toString()}]")

Compute filter (SLOW OPERATION even ~8 minutes!)

In [ ]:
GB=1024*1024*1024
db.computeFilter(db.getField(), 1*GB)
print("Filter computed")

In [ ]:
print("Compressing data...")
db.compressDataset("zip") 
print("compress dataset done")

Show subsampled and max:

In [ ]:
db_id =ov.LoadDataset('/mnt/c/big/visus-datasets/signal1d_slac/visus.idx'    )
db_max=ov.LoadDataset('/mnt/c/big/visus-datasets/signal1d_slac_max/visus.idx')
logic_box=db_id.getLogicBox()
resolution=8
data_sub=db_id .read(logic_box=logic_box, max_resolution=resolution)
data_max=db_max.read(logic_box=logic_box, max_resolution=resolution)[:,0]
print(f"read id  dtype={data_sub.dtype } shape={data_sub.shape } vmin={np.min(data_sub) } vmax={np.max(data_sub )}")
print(f"read max dtype={data_max.dtype} shape={data_max.shape} vmin={np.min(data_max)} vmax={np.max(data_max)}")
p = bokeh.plotting.figure(title="Simple line example", x_axis_label='x', y_axis_label='y')
p.line(list(range(len(data_sub))), data_sub, legend_label="sub", line_width=1, color="blue" );p.circle(list(range(len(data_sub))),data_sub, color="blue" )
p.line(list(range(len(data_max))), data_max, legend_label="max", line_width=1, color="green");p.circle(list(range(len(data_max))),data_max, color="green")
bokeh.io.show(p, notebook_handle=True)

Copy blocks to S3. For example:


```bash
aws s3 sync --no-verify-ssl --endpoint-url https://maritime.sealstorage.io/api/v0/s3 --profile sealstorage --size-only  C:/big/visus-datasets/signal1d_slac_max/ s3://utah/visus-datasets/signal1d_slac_max/
```